In [16]:
import networkx as nx
import igraph
import pandas as pd
import pylab as plt

In [17]:
print(Bio.__version__)

1.79


# age gene classes из Liebeskind

Скачать [age gene classes](https://github.com/marcottelab/Gene-Ages) из Liebeskind et. al. 2016, и замапить их на наши сети.\

In [92]:
yeast_data = pd.read_csv('../data/yeast.txt', sep='\t')
print(yeast_data.shape)

yeast_graph = nx.from_pandas_edgelist(yeast_data, source='Tf', target='Tg', create_using=nx.DiGraph())

print('# of edges: {}'.format(yeast_graph.number_of_edges()))
print('# of nodes: {}'.format(yeast_graph.number_of_nodes()))

(12873, 2)
# of edges: 12873
# of nodes: 4441


In [93]:
mouse_data = pd.read_csv('../data/trrust_rawdata.mouse.tsv', 
                         sep='\t', header=None, 
                         names=['TF', 'Target', 'Mode', 'PMID'])
print(mouse_data.shape)

mouse_graph = nx.from_pandas_edgelist(mouse_data, 'TF', 'Target', ['Mode', 'PMID'], create_using=nx.DiGraph())
print('# of edges: {}'.format(mouse_graph.number_of_edges()))
print('# of nodes: {}'.format(mouse_graph.number_of_nodes()))

(7057, 4)
# of edges: 6490
# of nodes: 2456


In [94]:
human_data = pd.read_csv('../data/trrust_rawdata.human.tsv', 
                         sep='\t', header=None, 
                         names=['TF', 'Target', 'Mode', 'PMID'])
print(human_data.shape)

human_graph = nx.from_pandas_edgelist(human_data, 'TF', 'Target', ['Mode', 'PMID'], create_using=nx.DiGraph())

print('# of edges: {}'.format(human_graph.number_of_edges()))
print('# of nodes: {}'.format(human_graph.number_of_nodes()))

(9396, 4)
# of edges: 8427
# of nodes: 2862


In [137]:
human_data

,TF,Target,Mode,PMID
0,AATF,BAX,Repression,22909821
1,AATF,CDKN1A,Unknown,17157788
2,AATF,KLK3,Unknown,23146908
3,AATF,MYC,Activation,20549547
4,AATF,TP53,Unknown,17157788
...,...,...,...,...
9391,ZNF76,CDKN1A,Repression,15280358
9392,ZNF76,PCYT1A,Activation,14702349
9393,ZNF76,TALDO1,Unknown,14702349
9394,ZNRD1,ABCB1,Activation,16373708


# Для начала можно просто посчитать количество пар TF-TG для каждого класса, нормированное на общее количество линков.



## Import classes from [Gene-Ages/Main/](https://github.com/marcottelab/Gene-Ages/tree/master/Main)

### Yeast

In [95]:
yeast_classes = pd.read_csv('../data/main_YEAST.csv')
print(yeast_classes.shape)
yeast_classes = yeast_classes.rename(columns={"Unnamed: 0": "Protein"})
yeast_classes.head()

(5000, 16)


,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Dikarya,Ascomycota,Saccharomyceta,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag
0,A2P2R3,0.800,0.0,0.2,0.000000,0.0,0.000000,0.0,0.000000,Cellular_organisms,5,1,0.500402,0.0,0.000000,False
1,D6VTK4,0.000,0.0,0.0,0.000000,0.0,0.000000,0.9,0.100000,Ascomycota,10,0,0.325083,0.2,0.107143,False
2,D6W196,0.000,0.0,0.0,0.500000,0.0,0.300000,0.0,0.200000,Eukaryota,10,2,1.029653,2.86363636364,2.236866,False
3,O13297,0.000,0.0,0.0,0.272727,0.0,0.454545,0.0,0.272727,Dikarya,11,0,1.067090,2.94545454545,0.120000,False
4,O13516,0.375,0.5,0.0,0.125000,0.0,0.000000,0.0,0.000000,Euk_Archaea,8,1,0.974315,0.777777777778,-0.220779,False


In [96]:
# for uniprot mapping - https://www.uniprot.org/uploadlists/

yeast_ID_UniProt = list(yeast_classes['Protein'])

with open('../data/age_classes/yeast_ID_UniProt.txt', 'w') as f:
    for el in yeast_ID_UniProt:
        f.write(el+' ')

 ... https://www.uniprot.org/uploadlists/ ...

In [97]:
yeast_prot_genes = pd.read_csv('../data/age_classes/Yeast_UniProtMapping.txt',
                                  sep='\t',
                                  engine='python',
                                  names=['Protein', 'Gene'],
                                  header=1)
yeast_prot_genes.head()

,Protein,Gene
0,D6VTK4,STE2
1,D6W196,SAL1
2,O13297,CET1
3,O13516,RPS9A
4,O13525,COQ4


In [99]:
yeast_classes = yeast_classes.merge(yeast_prot_genes, how='outer', on=['Protein'])
yeast_classes.head()

,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Dikarya,Ascomycota,Saccharomyceta,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag,Gene
0,A2P2R3,0.800,0.0,0.2,0.000000,0.0,0.000000,0.0,0.000000,Cellular_organisms,5,1,0.500402,0.0,0.000000,False,NaN
1,D6VTK4,0.000,0.0,0.0,0.000000,0.0,0.000000,0.9,0.100000,Ascomycota,10,0,0.325083,0.2,0.107143,False,STE2
2,D6W196,0.000,0.0,0.0,0.500000,0.0,0.300000,0.0,0.200000,Eukaryota,10,2,1.029653,2.86363636364,2.236866,False,SAL1
3,O13297,0.000,0.0,0.0,0.272727,0.0,0.454545,0.0,0.272727,Dikarya,11,0,1.067090,2.94545454545,0.120000,False,CET1
4,O13516,0.375,0.5,0.0,0.125000,0.0,0.000000,0.0,0.000000,Euk_Archaea,8,1,0.974315,0.777777777778,-0.220779,False,RPS9A


In [121]:
yeast_classes.shape

(5005, 17)

In [100]:
yest_TFTG = set(list(yeast_data.Tf) + list(yeast_data.Tg))

In [120]:
yeast_gene_ages = yeast_classes.query('Gene in @yest_TFTG')
#yeast_gene_ages = yeast_gene_ages[['Gene', 'modeAge']]
print(yeast_gene_ages.shape)
yeast_gene_ages.head()

(236, 17)


,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Dikarya,Ascomycota,Saccharomyceta,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag,Gene
318,P0CD98,0.125000,0.000000,0.375000,0.375000,0.125000,0.0,0.0,0.000000,Euk+Bac,8,2,1.255482,2.31111111111,1.380952,False,YLL053C
338,P10356,0.000000,0.076923,0.769231,0.000000,0.153846,0.0,0.0,0.000000,Euk+Bac,13,0,0.687092,1.51282051282,-0.234127,True,YER152C
807,P25351,0.076923,0.000000,0.307692,0.538462,0.000000,0.0,0.0,0.076923,Eukaryota,13,0,1.090599,2.4358974359,-0.067460,False,YCR023C
854,P25565,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.0,0.857143,Saccharomyceta,7,1,0.410116,2.25,0.583333,False,YCL002C
873,P25606,0.000000,0.000000,0.000000,0.666667,0.333333,0.0,0.0,0.000000,Eukaryota,3,1,0.636514,1.16666666667,NaN,False,YCR100C


#### Add New info to Graf:

### Mouse

In [109]:
mouse_classes = pd.read_csv('../data/main_MOUSE.csv')
print(mouse_classes.shape)
mouse_classes = mouse_classes.rename(columns={"Unnamed: 0": "Protein"})
mouse_classes.head()

(19950, 16)


,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Eumetazoa,Vertebrata,Mammalia,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag
0,A0A0A0MQ97,0.000000,0.000000,0.000000,0.0,0.0,0.375000,0.625000,0.000000,Vertebrata,8,5,0.661563,3.33333333333,1.599206,False
1,A0A0A0MQB3,0.333333,0.083333,0.333333,0.0,0.0,0.250000,0.000000,0.000000,Cellular_organisms,12,0,1.286057,3.24242424242,-0.166667,True
2,A0A0A0MQC1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.800000,0.200000,Vertebrata,5,1,0.500402,1.0,NaN,False
3,A0A0A0MQF7,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,Eumetazoa,2,0,-0.000000,2.0,NaN,False
4,A0A0B4J1F6,0.000000,0.000000,0.000000,0.0,0.0,0.142857,0.428571,0.428571,Mammalia,7,5,1.004242,4.16666666667,4.400000,False


In [110]:
mouse_TFTG = set(list(mouse_data.TF) + list(mouse_data.Target))

In [111]:
# for uniprot mapping - https://www.uniprot.org/uploadlists/

mouse_ID_UniProt = list(mouse_classes['Protein'])

with open('../data/age_classes/mouse_ID_UniProt.txt', 'w') as f:
    for el in mouse_ID_UniProt:
        f.write(el+' ')

 ... https://www.uniprot.org/uploadlists/ ...

In [112]:
mouse_prot_genes = pd.read_csv('../data/age_classes/mouse_UniProtMapping.txt',
                                  sep='\t',
                                  engine='python',
                                  names=['Protein', 'Gene'],
                                  header=1)
mouse_prot_genes.head()

,Protein,Gene
0,A0A0A0MQC1,Ighv3-5
1,A0A0A0MQF7,1700025G04Rik
2,A0A0B4J1F6,BC053393
3,A0A0B4J1F8,Smim10l2a
4,A0A0B4J1F9,Tchh


In [113]:
mouse_classes = mouse_classes.merge(mouse_prot_genes, how='outer', on=['Protein'])
mouse_classes.head()

,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Eumetazoa,Vertebrata,Mammalia,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag,Gene
0,A0A0A0MQ97,0.000000,0.000000,0.000000,0.0,0.0,0.375000,0.625000,0.000000,Vertebrata,8,5,0.661563,3.33333333333,1.599206,False,NaN
1,A0A0A0MQB3,0.333333,0.083333,0.333333,0.0,0.0,0.250000,0.000000,0.000000,Cellular_organisms,12,0,1.286057,3.24242424242,-0.166667,True,NaN
2,A0A0A0MQC1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.800000,0.200000,Vertebrata,5,1,0.500402,1.0,NaN,False,Ighv3-5
3,A0A0A0MQF7,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,Eumetazoa,2,0,-0.000000,2.0,NaN,False,1700025G04Rik
4,A0A0B4J1F6,0.000000,0.000000,0.000000,0.0,0.0,0.142857,0.428571,0.428571,Mammalia,7,5,1.004242,4.16666666667,4.400000,False,BC053393


In [116]:
mouse_classes.shape

(20001, 17)

In [117]:
mouse_gene_ages = mouse_classes.query('Gene in @mouse_TFTG')
#mouse_gene_ages = mouse_gene_ages[['Gene', 'modeAge']]
print(mouse_gene_ages.shape)
mouse_gene_ages.head()

(2287, 17)


,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Eumetazoa,Vertebrata,Mammalia,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag,Gene
44,A1L358,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,Opisthokonta,1,11,-0.000000,5.98484848485,3.577778,False,Mzf1
80,A2A5Z6,0.0,0.0,0.0,0.714286,0.0,0.142857,0.142857,0.000000,Eukaryota,7,5,0.796312,4.10606060606,4.533333,False,Smurf2
87,A2A6M5,0.0,0.0,0.0,0.090909,0.0,0.272727,0.363636,0.272727,Vertebrata,11,1,1.294545,5.37878787879,2.711111,False,Calcoco2
97,A2A7F4,0.0,0.0,0.0,0.000000,0.0,0.384615,0.615385,0.000000,Vertebrata,13,0,0.666278,1.02564102564,0.150794,False,Rlf
114,A2A863,0.0,0.0,0.0,0.000000,0.0,0.583333,0.416667,0.000000,Eumetazoa,12,1,0.679193,2.48717948718,1.988095,False,Itgb4


#### Add New info to Graf:

### Human

In [145]:
human_classes = pd.read_csv('../data/main_HUMAN.csv')
print(human_classes.shape)
human_classes = human_classes.rename(columns={"Unnamed: 0": "Protein"})
human_classes.head()

(20331, 16)


,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Eumetazoa,Vertebrata,Mammalia,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag
0,A0A075B6G5,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,Eumetazoa,1,1,-0.000000,6.0,NaN,False
1,A0A075B6R3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,Mammalia,2,1,-0.000000,10.6666666667,16.0,False
2,A0A0A0MR89,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,Eumetazoa,1,1,-0.000000,1.0,NaN,False
3,A0A0A0MS98,0.0,0.0,0.0,0.0,0.0,0.454545,0.0,0.545455,Mammalia,11,1,0.689009,5.68181818182,1.8,False
4,A0A0A0MSJ3,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,Eukaryota,2,0,-0.000000,1.0,NaN,False


In [146]:
human_TFTG = set(list(human_data.TF) + list(human_data.Target))

In [147]:
# for uniprot mapping - https://www.uniprot.org/uploadlists/

human_ID_UniProt = list(human_classes['Protein'])

with open('../data/age_classes/human_ID_UniProt.txt', 'w') as f:
    for el in human_ID_UniProt:
        f.write(el+' ')

 ... https://www.uniprot.org/uploadlists/ ...

In [148]:
human_prot_genes = pd.read_csv('../data/age_classes/human_UniProtMapping.txt',
                                  sep='\t',
                                  engine='python',
                                  names=['Protein', 'Gene'],
                                  header=1)
print(human_prot_genes.shape)
human_prot_genes.head()

(19629, 2)


,Protein,Gene
0,A0A075B6R3,HELZ
1,A0A0A0MR89,NRXN3
2,A0A0A0MS98,SLC4A1
3,A0A0A0MSL8,RBM14-RBM4
4,A0A0A0MTC1,RNF213


In [149]:
human_classes = human_classes.merge(human_prot_genes, how='outer', on=['Protein'])
human_classes.head()

,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Eumetazoa,Vertebrata,Mammalia,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag,Gene
0,A0A075B6G5,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,Eumetazoa,1,1,-0.000000,6.0,NaN,False,NaN
1,A0A075B6R3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,Mammalia,2,1,-0.000000,10.6666666667,16.0,False,HELZ
2,A0A0A0MR89,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,Eumetazoa,1,1,-0.000000,1.0,NaN,False,NRXN3
3,A0A0A0MS98,0.0,0.0,0.0,0.0,0.0,0.454545,0.0,0.545455,Mammalia,11,1,0.689009,5.68181818182,1.8,False,SLC4A1
4,A0A0A0MSJ3,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,Eukaryota,2,0,-0.000000,1.0,NaN,False,NaN


In [150]:
human_prot_genes = human_classes.query('Gene in @human_TFTG')
#human_prot_genes = human_prot_genes[['Gene', 'modeAge']]
print(human_prot_genes.shape)
human_prot_genes

(2855, 17)


,Protein,Cellular_organisms,Euk_Archaea,Euk+Bac,Eukaryota,Opisthokonta,Eumetazoa,Vertebrata,Mammalia,modeAge,NumDBsContributing,NumDBsFiltered,entropy,NodeError,Bimodality,HGT_flag,Gene
23,A0AVK6,0.0,0.0,0.000000,0.700000,0.000000,0.000000,0.300000,0.0,Eukaryota,10,3,0.610864,4.51282051282,2.861111,False,E2F8
94,A2AB90,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,Vertebrata,6,1,-0.000000,1.14285714286,4.000000,False,TAPBP
96,A2ADX3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,Vertebrata,4,1,-0.000000,0.0,NaN,False,HLA-DQB2
196,A6H8Y1,0.0,0.0,0.000000,0.666667,0.000000,0.333333,0.000000,0.0,Eukaryota,9,4,0.636514,6.35897435897,0.357143,False,BDP1
501,A6NNB3,0.0,0.0,0.000000,0.000000,0.000000,0.083333,0.916667,0.0,Vertebrata,12,1,0.286836,0.74358974359,0.091270,False,IFITM5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20459,Q9Y6W6,0.0,0.0,0.142857,0.714286,0.000000,0.142857,0.000000,0.0,Eukaryota,7,6,0.796312,3.58974358974,0.992063,False,DUSP10
20461,Q9Y6X0,0.0,0.0,0.000000,0.166667,0.000000,0.000000,0.833333,0.0,Vertebrata,12,1,0.450561,3.23076923077,0.738095,False,SETBP1
20463,Q9Y6X2,0.0,0.0,0.000000,0.583333,0.083333,0.000000,0.333333,0.0,Eukaryota,12,1,0.887694,4.64102564103,3.511905,False,PIAS3
20468,Q9Y6X8,0.0,0.0,0.000000,0.000000,0.000000,0.636364,0.363636,0.0,Eumetazoa,11,2,0.655482,1.07692307692,2.000000,False,ZHX2


#### Add New info to Graf:

## Более продвинутые идеи (например посмотреть коррелирует ли node degree с age class) уже тоже можно пробовать.